In [ ]:
# import sys
# sys.path.append('..')
# import utils.high_recall_information_retrieval as hrir

# system = hrir.HRSystem(from_scratch=False, 
#                   session_name='default_simulation', 
#                   finish_function=None, 
#                   debug=False,
#                   print_function=None)

# import numpy as np
# from sklearn.neural_network import MLPClassifier
# from utils.classifier import Classifier
# from utils.term_highlighter import TermHighlighter
# from utils.data_item import DataItem

# classifiers = [
# #                 Classifier(MLPClassifier(early_stopping=True,
# #                                          n_iter_no_change=20,
# #                                          max_iter=1500,
# #                                          hidden_layer_sizes=(20,), 
# #                                          solver='adam', 
# #                                          random_state=np.random.RandomState(42)), type_=DataItem.TYPE_BOW),
# #                 Classifier(MLPClassifier(early_stopping=True,
# #                                          n_iter_no_change=20,
# #                                          max_iter=1500,
# #                                          hidden_layer_sizes=(100,), 
# #                                          solver='adam', 
# #                                          random_state=np.random.RandomState(42)), type_=DataItem.TYPE_GLOVE300),
#               ]

# term_highlighter = None#TermHighlighter()

# models = []
# if not term_highlighter:
#     models = [term_highlighter]
# models += classifiers

# def argrank_function(yhat):
#     return list(reversed(np.argsort(yhat)))

# import pandas as pd
# if '__file__' in locals():
#     print(__file__)
#     output = '../simulation_results/' + __file__.split('/')[-1][:-3]+'.csv'
# else:
#     output = '../simulation_results/simulation_rsults_notebook.csv'

# simulation_results = system.simulation(arg_rank_function=argrank_function,
#                                        models=models,
#                                        starting_no_of_examples=5,
#                                        labeling_batch_size=10,
#                                        random_state=np.random.default_rng(2022))

# df = pd.DataFrame(simulation_results)
# df.to_csv(output)

In [1]:
import sys
print('starting simulation')
sys.path.append('/home/ec2-user/SageMaker/mariano/notebooks/07. Simulation/')
from utils.term_highlighter import TermHighlighter

from sklearn.neural_network import MLPClassifier
from utils.data_item import DataItem
from utils.classifier import Classifier
import pandas as pd
import numpy as np

# import utils.high_recall_information_retrieval as hrir
from utils.simulation import Simulator
from transformers import TrainingArguments
# from utils.classifier import HFModel

script_name='noname.py'
    
if '__file__' in locals():
    script_name = __file__.split('/')[-1]
    
output_name = script_name[:-3]+'.csv'

for split in range(1,6):
    input_split = f'/home/ec2-user/SageMaker/mariano/data/simulations/data/split_{split}/'

    # # # # # # # #
    # TRAIN DATA  #
    # # # # # # # #
    training_lines = open(input_split+'train.csv', 'r').read().splitlines()[1:]
    traininig = []
    for line in training_lines:
        id_, label = line.split(';')
        item = DataItem(id_)
        if label=='R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'
        traininig.append(item)
    # # # # # # # #
    # TRAIN DATA  #
    # # # # # # # #
    testing_lines = open(input_split+'test.csv', 'r').read().splitlines()[1:]
    testing = []
    for line in testing_lines:
        id_, label = line.split(';')
        item = DataItem(id_)
        if label=='R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'
        testing.append(item)
    print(len(testing))
    print(len(traininig))
    
    # # # # # # # #
    # HRIR System #
    # # # # # # # #

#     system = hrir.HRSystem(from_scratch=False, 
#                       session_name='default_simulation', 
#                       finish_function=None, 
#                       debug=False,
#                       print_function=None)
    simulator = Simulator()

    # # # # # # # # # # # # #
    # Arg Ranking Function  #
    # # # # # # # # # # # # #
    def relevance_argrank_function(yhat):
        return list(reversed(np.argsort(yhat)))
    
    def random_argrank_function(yhat):
        args = list(range(len(yhat)))
        np.random.shuffle(args)
        return args
    
    def uncertainty_argrank_function(yhat):
        return np.argsort(np.abs(np.array(yhat)-0.5)/0.5)


    # # # # # #
    # Models  #
    # # # # # #
    training_args = TrainingArguments("test-trainer")

    classifiers = [
#         HFModel(model_name='distilbert-base-uncased', training_args=training_args)
#                     Classifier(MLPClassifier(early_stopping=True,
#                                              n_iter_no_change=20,
#                                              max_iter=1500,
#                                              hidden_layer_sizes=(20,), 
#                                              solver='adam', 
#                                              random_state=np.random.RandomState(42)), type_=DataItem.TYPE_BOW),
        
#                     Classifier(MLPClassifier(early_stopping=True,
#                                              n_iter_no_change=20,
#                                              max_iter=1500,
#                                              hidden_layer_sizes=(100,), 
#                                              solver='adam', 
#                                              random_state=np.random.RandomState(42)), type_=DataItem.TYPE_GLOVE300),
              ]
    term_highlighter =TermHighlighter()

    models = []
    if not term_highlighter is None:
        models = [term_highlighter]

    models += classifiers
    
    # # # # # # # # # # # #
    # Running experiments #
    # # # # # # # # # # # #
    simulation_results = simulator.simulation(train_data=traininig,
                                              test_data=testing,
                                              arg_rank_function=relevance_argrank_function,
                                              models=models,
                                              starting_no_of_examples=5,
                                              labeling_batch_size=10,
                                              random_state=np.random.default_rng(2022))

    # # # # # # # # # #
    # Saving results  #
    # # # # # # # # # #
#     output_file = f'/home/ec2-user/SageMaker/mariano/data/simulations/results/split_{split}/{output_name}'
    

    
#     df = pd.DataFrame(simulation_results)
#     df.to_csv(output_file)


starting simulation
1457
5825
2022-08-01 11:51:07.022451 [ INFO  ] Number of labeled instances for training=          5,825
2022-08-01 11:51:07.022565 [ INFO  ] Number of labeled instances for testing=          1,457
2022-08-01 11:51:07.030766 [ INFO  ] len(unlabeled_data)=        5,820
2022-08-01 11:51:07.030859 [ INFO  ] len(labeled_data)=              5
2022-08-01 11:51:07.031143 [ INFO  ] len(rel labeled_data)=              5
2022-08-01 11:51:07.068231 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 0 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 11:51:07.068355 [ INFO  ] relevant count:   5
2022-08-01 11:51:07.068374 [ INFO  ] irrelevant count: 0
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>


/home/ec2-user/SageMaker/.conda/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/SageMaker/.conda/envs/bert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
2022-08-01 11:51:38.202400 [ INFO  ] AFTER LABELING
2022-08-01 11:51:38.202530 [ INFO  ] relevant count:   6
2022-08-01 11:51:38.202558 [ INFO  ] irrelevant count: 9
2022-08-01 11:51:38.237454 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 1 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 11:51:38.237578 [ INFO  ] relevant count:   6
2022-08-01 11:51:38.238007 [ INFO  ] irrelevant count: 9
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., 

computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
2022-08-01 11:52:05.376408 [ INFO  ] AFTER LABELING
2022-08-01 11:52:05.376577 [ INFO  ] relevant count:   58
2022-08-01 11:52:05.376622 [ INFO  ] irrelevant count: 47
2022-08-01 11:52:05.456749 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 10 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 11:52:05.456930 [ INFO  ] relevant count:   58
2022-08-01 11:52:05.456973 [ INFO  ] irrelevant count: 47
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, 

computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
2022-08-01 11:52:32.686151 [ INFO  ] AFTER LABELING
2022-08-01 11:52:32.686369 [ INFO  ] relevant count:   112
2022-08-01 11:52:32.686449 [ INFO  ] irrelevant count: 83
2022-08-01 11:52:32.793742 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 19 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 11:52:32.793973 [ INFO  ] relevant count:   112
2022-08-01 11:52:32.794044 [ INFO  ] irrelevant count: 83
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0955749270) trained=True vocab=<title

...

computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
2022-08-01 15:34:13.783410 [ INFO  ] AFTER LABELING
2022-08-01 15:34:13.785257 [ INFO  ] relevant count:   535
2022-08-01 15:34:13.787229 [ INFO  ] irrelevant count: 4880
2022-08-01 15:34:16.866210 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 541 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 15:34:16.868877 [ INFO  ] relevant count:   535
2022-08-01 15:34:16.871472 [ INFO  ] irrelevant count: 4880
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<

computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
2022-08-01 15:35:07.318902 [ INFO  ] AFTER LABELING
2022-08-01 15:35:07.322754 [ INFO  ] relevant count:   536
2022-08-01 15:35:07.325603 [ INFO  ] irrelevant count: 4969
2022-08-01 15:35:10.441134 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 550 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 15:35:10.442992 [ INFO  ] relevant count:   536
2022-08-01 15:35:10.444886 [ INFO  ] irrelevant count: 4969
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<

computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
2022-08-01 15:36:01.691414 [ INFO  ] AFTER LABELING
2022-08-01 15:36:01.693380 [ INFO  ] relevant count:   536
2022-08-01 15:36:01.694961 [ INFO  ] irrelevant count: 5059
2022-08-01 15:36:04.639417 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 559 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 15:36:04.641353 [ INFO  ] relevant count:   536
2022-08-01 15:36:04.643002 [ INFO  ] irrelevant count: 5059
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<

computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
2022-08-01 15:36:57.363889 [ INFO  ] AFTER LABELING
2022-08-01 15:36:57.366351 [ INFO  ] relevant count:   538
2022-08-01 15:36:57.368166 [ INFO  ] irrelevant count: 5147
2022-08-01 15:37:00.455344 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 568 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 15:37:00.457192 [ INFO  ] relevant count:   538
2022-08-01 15:37:00.458666 [ INFO  ] irrelevant count: 5147
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<

computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
2022-08-01 15:37:52.447343 [ INFO  ] AFTER LABELING
2022-08-01 15:37:52.449335 [ INFO  ] relevant count:   538
2022-08-01 15:37:52.450981 [ INFO  ] irrelevant count: 5237
2022-08-01 15:37:55.537516 [ INFO  ]  ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ LOOP NO. 577 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
2022-08-01 15:37:55.541246 [ INFO  ] relevant count:   538
2022-08-01 15:37:55.544560 [ INFO  ] irrelevant count: 5237
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<title, ..., new>>
computing on model: <TermHighlighter model=LogisticRegression(C=1, random_state=RandomState(MT19937) at 0x7F0946C09AF0) trained=True vocab=<

In [ ]:
df  = pd.DataFRame(simulation_results)

In [3]:
# import numpy as np
# def uncertainty_argrank_function(yhat):
#     return np.argsort(np.abs(np.array(yhat)-0.5)/0.5)
# uncertainty_argrank_function([0,0,0.5,0.6,0.8,0])
# #2, 3, 4, 1,2,5

array([2, 3, 4, 0, 1, 5])

In [1]:
import spacy

In [ ]:
# import sys
# sys.path.append('..')
# import utils.high_recall_information_retrieval as hrir
# system = hrir.HRSystem(from_scratch=False, 
#                   session_name='default_simulation', 
#                   finish_function=None, 
#                   debug=False,
#                   print_function=None)

In [ ]:
# from utils.classifier import HFModel
# model = HFModel('bert-base-uncased')

In [ ]:
# model.fit(system.labeled_data[:10])

In [ ]:
# TIEMPO: 10 instancias, 15 epocas (30 optimization steps), 337segs (5min y 37 segs)

In [ ]:
# from utils.data_item import DataItem
# DataItem.get_y(system.labeled_data)

In [ ]:
# import numpy as np 
# y=DataItem.get_y(system.labeled_data)
# new_y = np.zeros(shape=(len(y),2))
# for idx, arg in enumerate(y):
#     print(idx)
#     print(arg)
#     new_y[idx,int(arg)]=1